In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v1 as tf

from glob import glob
from re import findall,sub
from tqdm.notebook import tqdm

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import  *
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

%matplotlib inline

/home/codekage/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/codekage/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/codekage/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/codekage/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [2]:
tf.enable_eager_execution()

In [3]:
tf.config.experimental_list_devices()

['/job:localhost/replica:0/task:0/device:CPU:0',
 '/job:localhost/replica:0/task:0/device:XLA_GPU:0',
 '/job:localhost/replica:0/task:0/device:XLA_CPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
images = glob("./lfw_funneled/*/*")

In [5]:
people = np.array([[" ".join(i.split("/")[-1].split("_")[:-1]),i] for i in images])

### Counting Occurence Of A Person

In [6]:
from collections import Counter
from random import choice

import pandas as pd

In [7]:
face_occurence = Counter(people[:,0])

In [8]:
face_occurence = pd.DataFrame([{"name":i,"occurence":face_occurence[i]} for i in face_occurence])

In [9]:
face_occurence.head()

,name,occurence
0,Dita Von Tesse,1
1,Chris Byrd,2
2,Irfan Ahmed,1
3,John Edwards,8
4,Manfred Reyes Villa,1


In [334]:
names_to_use = face_occurence.name[face_occurence.occurence > 1]

In [335]:
haar = cv2.CascadeClassifier()
haar.load("../haarcascade_frontalface_default.xml")

True

In [336]:
# names = []
# images = []
# failed = []

# for name,path in tqdm(people):
#     img = cv2.imread(path)
#     try:
#         gray = img.mean(axis=2).astype(np.uint8)
#         x,y,w,h = haar.detectMultiScale(gray)[0]
#         img = cv2.resize(img[y:y+h,x:x+w],(128,128))
#         names.append(name)
#         images.append(img)
#     except Exception as e:
#         failed.append([name,img,e])
    
# images = np.array(images)
# names = np.array(names)

# np.save("./names.npy",names)
# np.save("./faces.npy",images)

In [337]:
names = np.load("./names.npy")
images = np.load("./faces.npy").astype(np.float32)
images= images / 255

In [352]:
pairs = []

for name in set(names_to_use):
    group = np.where(name == names)[0]
    l = len(group)
    for i in range(l):
        pairs.append([
            group[i],
            group[(i+1)%l],
            np.random.randint(13190)
        ])

pairs = np.array(pairs)

In [353]:
pairs[:10]

array([[ 9460,  9461,  4698],
       [ 9461,  9462, 11644],
       [ 9462,  9463, 12558],
       [ 9463,  9460,  3211],
       [ 7404,  7405,  9001],
       [ 7405,  7404,  4367],
       [10253, 10254, 12505],
       [10254, 10253,  2875],
       [ 1272,  1273,  6052],
       [ 1273,  1274,  1745]])

In [465]:
model = keras.Sequential([
    keras.layers.Conv2D(64,4,activation="relu",input_shape=(128,128,3)),
    keras.layers.Conv2D(64,3,activation="relu"),
    keras.layers.Conv2D(64,3,activation="relu"),
    keras.layers.MaxPool2D(),
    
    keras.layers.Conv2D(64,3,activation="relu"),
    keras.layers.Conv2D(64,3,activation="relu"),
    keras.layers.Conv2D(64,3,activation="relu"),
    keras.layers.MaxPool2D(),
    
    keras.layers.Conv2D(128,3,activation="relu"),
    keras.layers.Conv2D(128,3,activation="relu"),
    keras.layers.Conv2D(128,3,activation="relu"),
    keras.layers.MaxPool2D(),
    
    keras.layers.Conv2D(128,3,activation="relu"),
    keras.layers.Conv2D(128,3,activation="relu"),
    keras.layers.Conv2D(128,3,activation="relu"),
    keras.layers.MaxPool2D(),
        
    keras.layers.Flatten(),
    keras.layers.Dense(2048,activation="relu"),
    keras.layers.Dense(1024,activation="relu"),
    keras.layers.Dense(128,activation="linear"),
    keras.layers.BatchNormalization()
])

anchor = keras.layers.Input(shape=(128,128,3),name="anchor")
positive = keras.layers.Input(shape=(128,128,3),name="positive")
negative = keras.layers.Input(shape=(128,128,3),name="negative")

a_out = model(anchor)
p_out = model(positive)
n_out = model(negative)

out = keras.layers.concatenate([a_out,p_out,n_out],axis=-1)
model_train = keras.models.Model([anchor,positive,negative],out)

In [466]:
model_train.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor (InputLayer)             [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
positive (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
negative (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
sequential_19 (Sequential)      (None, 128)          4280064     anchor[0][0]                     
                                                                 positive[0][0]             

![triplet_loss](./triplet_function.png)

In [526]:
class TripletLoss:    
    def __init__(self,margin=0.1):
        self.margin = margin
        pass
    
    def __call__(self,y_true,y_pred=None,**kwargs):
        a = y_true[:,0::3]
        p = y_true[:,1::3]
        n = y_true[:,2::3]

        pos = keras.backend.sqrt(keras.backend.sum(keras.backend.square(p-a),axis=1))
        neg = keras.backend.sqrt(keras.backend.sum(keras.backend.square(n-a),axis=1))
        
        return tf.maximum(pos - neg,0)

In [527]:
train = pairs.reshape(-1,1)

In [528]:
def input_func(x,batch_size=1,epochs=1):
    for epoch in range(epochs):
        x = x.reshape(-1,batch_size*3)
        for batch in x:
            batch = images[batch]
            yield {
                    "anchor":batch[::3],
                    "positive":batch[1::3],
                    "negative":batch[2::3]
                }

In [529]:
triplet_loss = TripletLoss()
opt = keras.optimizers.SGD()

In [536]:
batch_size = 11
epochs = 5
gen = input_func(train,batch_size=batch_size,epochs=epochs)


for epoch in range(10):
    epoch_losses = []
    for i in range(train.shape[0]//(3*batch_size)):
        with tf.GradientTape() as tape:
            x = next(gen)
            out = model_train(x)
            loss = triplet_loss(out)
            loss_ = keras.backend.mean(loss).numpy()
            epoch_losses.append(loss_)
            print (f"Epoch : {epoch+1} | Loss : {str(loss_)}")
        grad = tape.gradient(loss,model.trainable_variables)
        opt.apply_gradients(zip(grad,model.trainable_variables))        
        
#     print (f"Epoch : {epoch+1} | Loss : {str(keras.backend.mean(loss).numpy())}")

Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 | Loss : 0.0
Epoch : 1 |

KeyboardInterrupt: 